In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

In [2]:
# Initialize SparkSession
spark = SparkSession.builder.appName('25m_movie_lens').getOrCreate()

In [3]:
# Load both the ratings and movies data into a pandas DataFrame
ratings_df = spark.read.csv('./ml-25m/ratings.csv', header=True, inferSchema=True).limit(5000)
movies_df = spark.read.csv('./ml-25m/movies.csv', header=True, inferSchema=True).limit(5000)

ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [4]:
# Left join ratings on movies
join_df = ratings_df.join(movies_df, on='movieId', how='left') \
    .select(col('userId'), col('movieId'), col('rating'), col('title'), col('genres')) \
    .sort('userId')

join_df.show()

+------+-------+------+--------------------+--------------------+
|userId|movieId|rating|               title|              genres|
+------+-------+------+--------------------+--------------------+
|     1|    296|   5.0| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|     1|    306|   3.5|Three Colors: Red...|               Drama|
|     1|    307|   5.0|Three Colors: Blu...|               Drama|
|     1|    665|   5.0|  Underground (1995)|    Comedy|Drama|War|
|     1|    899|   3.5|Singin' in the Ra...|Comedy|Musical|Ro...|
|     1|   1088|   4.0|Dirty Dancing (1987)|Drama|Musical|Rom...|
|     1|   1175|   3.5| Delicatessen (1991)|Comedy|Drama|Romance|
|     1|   1217|   3.5|          Ran (1985)|           Drama|War|
|     1|   1237|   5.0|Seventh Seal, The...|               Drama|
|     1|   1250|   4.0|Bridge on the Riv...| Adventure|Drama|War|
|     1|   1260|   3.5|            M (1931)|Crime|Film-Noir|T...|
|     1|   1653|   4.0|      Gattaca (1997)|Drama|Sci-Fi|Thri...|
|     1|  

In [5]:
(train_df, test_df) = join_df.randomSplit([0.8, 0.2])

In [6]:
# Define ALS model
als = ALS(maxIter=20, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True)

In [7]:
# Fit data and make predictions
model = als.fit(train_df)
predictions = model.transform(test_df)

predictions.show()

+------+-------+------+--------------------+--------------------+----------+
|userId|movieId|rating|               title|              genres|prediction|
+------+-------+------+--------------------+--------------------+----------+
|    15|   1088|   2.0|Dirty Dancing (1987)|Drama|Musical|Rom...| 4.8114543|
|    18|   1580|   4.0|Men in Black (a.k...|Action|Comedy|Sci-Fi| 2.7868874|
|    30|   1580|   5.0|Men in Black (a.k...|Action|Comedy|Sci-Fi| 2.3709283|
|    30|   3175|   4.5| Galaxy Quest (1999)|Adventure|Comedy|...| 2.5057309|
|    31|   1580|   3.0|Men in Black (a.k...|Action|Comedy|Sci-Fi| 2.5196283|
|    31|   3175|   1.5| Galaxy Quest (1999)|Adventure|Comedy|...| 2.0850792|
|     2|    858|   3.5|Godfather, The (1...|         Crime|Drama| 4.8236017|
|     3|    858|   5.0|Godfather, The (1...|         Crime|Drama| 3.9883099|
|     3|   1127|   3.5|   Abyss, The (1989)|Action|Adventure|...| 4.0529284|
|     4|  48780|   5.0|                null|                null|   2.36669|

In [8]:
# Evaluate model using RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error = {:.4f}".format(rmse))

Root Mean Squared Error = 1.6842


In [9]:
def get_title(movie_id):
    return (movies_df.filter(movies_df.movieId == movie_id).collect()[0][1])

In [10]:
#Predict list of movies for a specific user
movie_ids = [1, 11, 111]
user_ids = [10, 10, 10]

user_preds = spark.createDataFrame(zip(movie_ids, user_ids), schema=['movieId', 'userId'])

preds = model.transform(user_preds)

In [ ]:
# Stop SparkSession
spark.stop()